In [2]:
### インポート ###
import numpy as np
import pandas as pd
import glob
from selenium.webdriver.chrome.service import Service
from datetime import datetime
import time
import re
import csv
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, TypeVar, Generic
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib
import tensorflow as tf

In [3]:
# =====
# 設定
# =====

### 設定ファイル ###
df_config_style = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="style", header=0)
df_config_target_courses = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="target_courses", header=0)
df_config_features = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="features", header=0)
df_config_model = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="data_model", header=0)
# 脚質
STYLE_MAP = df_config_style.set_index('key')['value'].to_dict()
print(f'脚質: {STYLE_MAP}')
REVERSE_STYLE_MAP = {v: k for k, v in STYLE_MAP.items()}
# モデル
MIN_RECORDS_MAP = df_config_model.set_index('num_horses')['min_records'].to_dict()
STD_MAP = df_config_model.set_index('num_horses')['rand_std'].to_dict()
print(f'最小許容有効レコード数: {MIN_RECORDS_MAP}')
print(f'乱数作成時の標準偏差: {STD_MAP}')
# 最小騎手出走数
FEATURES_MAP = df_config_features.set_index('key')['value'].to_dict()
MIN_JOCKEY_RECORDS = FEATURES_MAP.get('min_jockey_records')
print(f'最小騎手出走数: {MIN_JOCKEY_RECORDS}')
# 対象コース
# 前準備：Excelのコース設定を、高速に検索できる「集合(set)」形式に変換
target_courses_set = set(
    df_config_target_courses[['racecourse', 'ground', 'distance', 'direction']]
    .itertuples(index=False, name=None)
)
print(f'対象コース: {target_courses_set}')
### 共通オブジェクト ###
scaler = StandardScaler()

脚質: {1: '逃げ', 2: '先行', 3: '差し', 4: '追込'}
最小許容有効レコード数: {8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
乱数作成時の標準偏差: {8: 0.08, 9: 0.09, 10: 0.1, 11: 0.11, 12: 0.12}
最小騎手出走数: 100
対象コース: {('佐賀', 'ダ', 1300, '右'), ('姫路', 'ダ', 1400, '右'), ('高知', 'ダ', 1300, '右'), ('笠松', 'ダ', 1600, '右'), ('盛岡', 'ダ', 1200, '左'), ('園田', 'ダ', 1400, '右'), ('金沢', 'ダ', 1500, '右'), ('浦和', 'ダ', 1500, '左'), ('名古屋', 'ダ', 1500, '右'), ('船橋', 'ダ', 1500, '左'), ('佐賀', 'ダ', 1400, '右'), ('水沢', 'ダ', 850, '右'), ('園田', 'ダ', 1230, '右'), ('水沢', 'ダ', 1400, '右'), ('門別', 'ダ', 1000, '右'), ('高知', 'ダ', 1400, '右'), ('門別', 'ダ', 1200, '右'), ('川崎', 'ダ', 900, '左'), ('川崎', 'ダ', 1400, '左'), ('笠松', 'ダ', 1400, '右'), ('船橋', 'ダ', 1200, '左'), ('浦和', 'ダ', 1400, '左')}


In [4]:
# ========
#メソッド
# ========

def create_dl_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid') 
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# =================
# 学習データ読み込み
# =================
print("trainデータ読み込み：開始")
df_train = pd.read_csv("C:\\keiba\\tool\\train\\train.csv", header=0, encoding='cp932')
print("trainデータ読み込み：終了")

trainデータ読み込み：開始
trainデータ読み込み：終了


In [10]:
# =======================
# モデル作成 / 学習 / 保存
# =======================

# 説明変数
features = [
    'time_index_pred_from_race_avg',
    f'jockey_top3_rate_recent_{MIN_JOCKEY_RECORDS}_from_race_avg',
    'condition_win_rate',
    'position_half_type_win_rate',
    'composition_pos_win_rate'
]

# 頭数ごとにモデル作成 / 学習
for num_horses in STD_MAP:

    print(f"\n--- 頭数 {num_horses} の処理を開始 ---")
    
    # 該当する頭数のデータを抽出
    train_sub = df_train[df_train['num_horses'] == num_horses].copy()
    
    ### モデル作成 ###
    print(f"モデル作成（頭数:{num_horses}）：開始")
    model = create_dl_model(len(features))
    print(f"モデル作成（頭数:{num_horses}）：終了")

    ### 学習 ###
    print(f"学習（頭数:{num_horses}）：開始")
    X_train = train_sub[features]
    y_train = train_sub['relative_rank_bin']
    X_train_scaled = scaler.fit_transform(X_train)
    model.fit(X_train_scaled, y_train, epochs=20, batch_size=16, verbose=0)
    print(f"学習（頭数:{num_horses}）：終了")

    ### 保存 ###
    model.save(f'C:\\keiba\\tool\\model\\model_{num_horses}_20260119.keras')
    joblib.dump(scaler, f'C:\\keiba\\tool\\model\\scaler_{num_horses}_20260119.joblib')



--- 頭数 8 の処理を開始 ---
モデル作成（頭数:8）：開始


C:\Users\ryo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


モデル作成（頭数:8）：終了
学習（頭数:8）：開始
学習（頭数:8）：終了

--- 頭数 9 の処理を開始 ---
モデル作成（頭数:9）：開始
モデル作成（頭数:9）：終了
学習（頭数:9）：開始


C:\Users\ryo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


学習（頭数:9）：終了

--- 頭数 10 の処理を開始 ---
モデル作成（頭数:10）：開始


...

モデル作成（頭数:10）：終了
学習（頭数:10）：開始
学習（頭数:10）：終了

--- 頭数 11 の処理を開始 ---
モデル作成（頭数:11）：開始


C:\Users\ryo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


モデル作成（頭数:11）：終了
学習（頭数:11）：開始
学習（頭数:11）：終了

--- 頭数 12 の処理を開始 ---
モデル作成（頭数:12）：開始


C:\Users\ryo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


モデル作成（頭数:12）：終了
学習（頭数:12）：開始
学習（頭数:12）：終了
